# Gremlin Workshop - Section 02: Find/Traversal Steps

Now that you have the basics down, let's start working on queries that are more related to traversing a graph structure.  In the following sections. You will use the Gremlin steps necessary for navigating a graph.  This includes crossing common edges and related vertices along a given traversal path.

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-updated.png"/>



## Following Edges

Storing relationships in the same manner as entities is a key differentiating factor for graph databases.  Moreso, traversing these relationships is what make graph databases powerful, unique, and fulfills the role of a "purpose-built" tool for querying highly connected datasets.  In the following exercises, we will use Gremlin steps to show how to traverse across relationships, or edges, in a graph.

### 02.01 - Using in()

Let's start with a simple traversal that crosses an edge.  In the depiction of the data model above, you will notice that Artists are connected to movies via edges that are labeled with their role, e.g. actor, writer, producer, director, or actress.  If we wanted to find all productions that a given Artist worked on, we can traverse across an edge with a given role to the movies related to that Artist.  Let's give this a try.  

In the following example, we will find the movies directed by Christopher Nolan. In this example, we start by finding the vertex representative of Christopher Nolan `g.V().hasLabel('Artist').has('name','Christopher Nolan')` and then we use the `in('director')` Gremlin step to traverse across the incoming edge with the label of _director_.  At this point, we now have a list of all of the vertices related to movies of which Christopher Nolan directed.  We then use the `values()` step to list out the titles of each of these productions.

__NOTE: Edges in Neptune have directionality.  You can use `in()`, `out()`, or `both()` to traverse across edges with certain directions.  The `both()` step ignores edge direction.  It is a best practice to always specify the edge label within these steps whenever possible.  In the the following example, we've done that using `in('director')`.__

Consult the following documentation:

 - [Some simple graph traversal examples](http://kelvinlawrence.net/book/Gremlin-Graph-Guide.html#_some_simple_graph_traversal_examples) describes how you can use `out()`, `in()` and `both()` to traverse to neighbouring vertices.
 - [Examining the edge between two vertices](http://kelvinlawrence.net/book/Gremlin-Graph-Guide.html#exedge) describes how you can use `outE()`, `inE()` and `bothE()` to examine the edges themselves.

In [ ]:
%%gremlin

g.V().hasLabel('Artist').has('name','Christopher Nolan').
    in('director').values('title')

### 02.02 - Using out()

Similarly to the example above, we can use the `out()` step to traverse across outgoing edges.  In this next example, we are using the Person and Artist vertices along with the Follows edges.  Run the following code block to find the person with ID 'person378' and count the number of people they follow.

We also introduce a new step in this example, `count()`.  Count does exactly as it implies.  It takes whatever input is mapped to it and outputs a count of those objects.

Consult the following documentation:

- [Counting Things](https://kelvinlawrence.net/book/Gremlin-Graph-Guide.html#count) - More examples for using `count()`

In [ ]:
%%gremlin

g.V('person378').
    out('follows').count()

### 02.03 - Using multiple in() and out() steps to traverse a graph

Here's an example where you can chain `in()` and `out()` steps together to navigate deeper into a graph.  In the following example, find the Person with ID 'person378' and count the number of unique people (themselves _included_) who follow the same artists.

You will notice that we're also using another new step here, `dedup()`.  Much like `count()`, `dedup()` does at it implies.  It removes deplicates that are mapped to it and returns a list of de-duplicated objects.  We need `dedup()` in this case as each Artist vertex that we traverse across may have corresponding Follow edge connections to some of the same people.  We don't care about the duplicates here, as we only want a count of the unique Person vertices that follow the same Artists as Person vertex 'person378'.

Consult the following documentation:
 - [`dedup()`](http://tinkerpop.apache.org/docs/current/reference/#dedup-step)

In [ ]:
%%gremlin

g.V('person378').
    out('follows').in('follows').
    dedup().count()

### 02.04 - Using as() and select()

In these exercises you'll use `as()` to label a step that you can refer back to in a subsequent step.  We'll also introduce `select()`.  The `select()` step can be used in a later part of a traversal to refer back to a label provided by `as()`.

See the following documentation:

 - [`as()`](http://tinkerpop.apache.org/docs/current/reference/#as-step) 
 - [`select()`](http://tinkerpop.apache.org/docs/current/reference/#select-step)
 
As an example, say we wanted to output all artist that co-starred with Halle Berry and the movies in which they appeared together. Let's try that via the following query:

In [ ]:
%%gremlin

g.V().hasLabel('Artist').has('name','Halle Berry').
    in('actor','actress').as('movie').
    out('actor','actress').as('costar').
    select('movie','costar').
    by('title').
    by('name')

Note that the `by()` steps after `select()` allow us to interact with the selected labels.  So in this case, `select()` returns the vertices associated with the movie titles and the vertices associated with the performers.  Instead of just returning the vertex IDs, we want to return some properties associated with them - movie title and performer name, respectively.

### 02.05 - Using where() to filter results

That last query was pretty cool, but it did include Halle in the results.  How can we get results without Halle listed and only her costars?  We can do this by adding a conditional and using the `where()` step.  The `where()` step acts as a filter and allows us to provide a conditional.  In this case, we can say "return all of these results where the costar does not equal 'Halle Berry'".

See the following documentation:

- [`where()`](http://tinkerpop.apache.org/docs/3.4.4/reference/#where-step)

In [ ]:
%%gremlin

g.V().hasLabel('Artist').has('name','Halle Berry').as('halle').
    in('actor','actress').as('movie').
    out('actor','actress').
    where(neq('halle')).as('costar').
    select('movie','costar').
    by('title').
    by('name')

### 02.07 - Using aggregate(), group().by(), and order().by()

Let's trying something a bit more complex.  Ever wondered how Facebook, Twitter, or LinkedIn provide you with a list of people that you "might want to connect with"?  This type of traversal can also be referred to as the __Friends of Friends__ (FoF) traversal. 

We can simulate FoF in our IMDb dataset by looking at co-stars of co-stars for a particular performer.  We can basically ask "what performers should our unique performer look to star with based on their total number of common co-stars".

In doing this type of query, we'll need to leverage the `aggregate()` step.  The `aggregate()` step creates a temporary collection within the middle of our traversal that we can refer back to.  In this case, we'll want to aggregate all co-stars of a given performer, then the co-stars of co-stars, and then count the number of times the co-stars of co-stars appear across common co-star connections.  We will also want to filter out co-stars and our given performer from the results.

See the following documentation:
- [`aggregate()`](http://tinkerpop.apache.org/docs/3.4.4/reference/#store-step)

In the following example, we'll use Rebel Wilson and find which performer she has not appeared with but shares the highest number of common co-stars.


In [ ]:
%%gremlin

g.V().hasLabel('Artist').has('name','Rebel Wilson').as('rebel').
    in('actor','actress').hasLabel('movie').out('actor','actress').aggregate('costars').
    in('actor','actress').hasLabel('movie').out('actor','actress').where(without('costars')).where(neq('rebel')).
    groupCount().by('name').unfold().
    order().by(values,desc)

In the above query, we also used a few new steps.  Much like in other database query languages, Gremlin provides a means to do grouping and ordering.  You can use `group().by()` or `groupCount().by()` for grouping.  As well, you can perform ordering of results using `order().by()`.   In both cases, the `by()` step determines which field of the results you are grouping and ordering.  The `by()` after order also takes the direction or ordering by using either `desc` or `asc`.

Lastly, we used the `unfold()` step. In this case, the `groupCount()` step returned a collection of results. In order to sort these, we needed to break each item in the collection into a separate element.  We used `unfold()` for this.  There is also a `fold()` step.  Both are very useful for managing collections and we'll see these more in other patterns later in this workshop.

See the following documentation:
- [`group()`](http://tinkerpop.apache.org/docs/3.4.4/reference/#group-step)
- [`groupCount()`](http://tinkerpop.apache.org/docs/3.4.4/reference/#groupcount-step)
- [`order()`](http://tinkerpop.apache.org/docs/3.4.4/reference/#order-step)
- [`fold()`](http://tinkerpop.apache.org/docs/3.4.4/reference/#fold-step)
- [`unfold()`](http://tinkerpop.apache.org/docs/3.4.4/reference/#unfold-step)

### Review

In the previous notebook we started using Gremlin to traverse across both vertices and edges in a graph.  We displayed how these queries can be used to take advantage of the first class representation of relationships in a graph.  Lastly, we also looked at different types of aggregations and how to display results in an ordered fashion.  We will continue building on the use of these patterns in the next notebook.

### End of Section 02 - Move onto the next notebook.